# Movielens 영화 추천 

- MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작
- 데이터셋은 추천 시스템의 MNIST라고 부를만한 Movielens 데이터임.

유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. MovieLens 1M Dataset 사용을 권장합니다.
별점 데이터는 대표적인 explicit 데이터입니다. 하지만 implicit 데이터로 간주하고 테스트해 볼 수 있습니다.
별점을 시청횟수로 해석해서 생각하겠습니다.
또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외하겠습니다.
Cloud Storage에 미리 업로드된 ml-1m폴더 내 파일을 심볼릭 링크로 개인 storage에 연결해 줍니다.

In [1]:
import numpy as np
import scipy
import implicit
import pandas as pd
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.4
1.7.1
0.4.8


## 1) 데이터 준비와 전처리

- Movielens 데이터는 rating.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있음. 

In [2]:
import os

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head(100)

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
95,2,2490,3,978299966
96,2,1834,4,978298813
97,2,3471,5,978298814
98,2,589,4,978299773


In [3]:
ratings = ratings[ratings['ratings']>=3] # 3점 이상만 남깁니다.
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [4]:
# ratings 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'ratings':'counts'}, inplace=True)

## 2) 데이터 분석


In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어오기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [7]:
num_movie = ratings['movie_id'].nunique()
num_user = ratings['user_id'].nunique()
print("ratings에 있는 유니크한 영화 개수: ",num_movie) 
print("ratings에 있는 유니크한 사용자 수: ", num_user)

ratings에 있는 유니크한 영화 개수:  3628
ratings에 있는 유니크한 사용자 수:  6039


In [8]:
ratings

,user_id,movie_id,counts,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000203,6040,1090,3,956715518
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [9]:
movie_data = pd.merge(movies, ratings)
movie_data.head()

,movie_id,title,genre,user_id,counts,timestamp
0,1,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268
1,1,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008
2,1,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496
3,1,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952
4,1,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474


In [10]:
#가장 인기있는 영화 30개(인기순)
movies_count = movie_data.groupby('title')['user_id'].count()
movies_count.sort_values(ascending=False).head(500)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
                                                         ... 
Home Alone (1990)                                         508
Hudsucker Proxy, The (1994)                               508
Moonraker (1979)                                          507
Jerk, The (1979)                                          504
Cliffhanger (1993)                                        504
Name: user_id, Length: 500, dtype: int64

In [11]:
# 유저별 몇 개의 영화를 보고 있는지에 대한 통계
user_count = movie_data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## 3) 내가 선호하는 영화 5가지 ratings에 추가

In [12]:
# 좋아하는 영화 5개 추가
my_favorite_movie = ['Back to the Future (1985)','Home Alone (1990)', 'Erin Brockovich (2000)', 'Jurassic Park (1993)', 'Sixth Sense, The (1999)']

my_playlist = pd.DataFrame({'user_id' : 6040, 'title': my_favorite_movie, 'counts':[5]*5})


movie_data  = movie_data .append(my_playlist, ignore_index=True)                        

movie_data  # 잘 추가되었는지 확인해 봅시다.

,movie_id,title,genre,user_id,counts,timestamp
0,1.0,Toy Story (1995),Animation|Children's|Comedy,1,5,978824268.0
1,1.0,Toy Story (1995),Animation|Children's|Comedy,6,4,978237008.0
2,1.0,Toy Story (1995),Animation|Children's|Comedy,8,4,978233496.0
3,1.0,Toy Story (1995),Animation|Children's|Comedy,9,5,978225952.0
4,1.0,Toy Story (1995),Animation|Children's|Comedy,10,5,978226474.0
...,...,...,...,...,...,...
836478,NaN,Back to the Future (1985),NaN,6040,5,NaN
836479,NaN,Home Alone (1990),NaN,6040,5,NaN
836480,NaN,Erin Brockovich (2000),NaN,6040,5,NaN
836481,NaN,Jurassic Park (1993),NaN,6040,5,NaN


In [13]:
# 사용하는 컬럼만 남겨줍니다.
movie_data = movie_data[['user_id','title', 'counts']]
movie_data.sort_index()

,user_id,title,counts
0,1,Toy Story (1995),5
1,6,Toy Story (1995),4
2,8,Toy Story (1995),4
3,9,Toy Story (1995),5
4,10,Toy Story (1995),5
...,...,...,...
836478,6040,Back to the Future (1985),5
836479,6040,Home Alone (1990),5
836480,6040,Erin Brockovich (2000),5
836481,6040,Jurassic Park (1993),5


In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = movie_data['user_id'].unique()
movie_unique = movie_data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = movie_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = movie_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movie_data):
    print('movie column indexing OK!!')
    movie_data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

movie_data

user_id column indexing OK!!
movie column indexing OK!!


/tmp/ipykernel_75/780136522.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다.
/tmp/ipykernel_75/780136522.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movie_data['title'] = temp_movie_data


,user_id,title,counts
0,0,0,5
1,1,0,4
2,2,0,4
3,3,0,5
4,4,0,5
...,...,...,...
836478,1999,1152,5
836479,1999,566,5
836480,1999,3110,5
836481,1999,462,5


## 4) CSR matrix 만들기

In [16]:
num_user = movie_data['user_id'].nunique()
num_movie = movie_data['title'].nunique()

csr_data = csr_matrix((movie_data.counts, (movie_data.user_id, movie_data.title)), shape=(num_user, num_movie)) 
csr_data

<6039x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836481 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델 구성, 훈련


In [17]:
# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언, iteration은 나중에 바꿔주겠습니다.
als_model = AlternatingLeastSquares(factors=100, regularization=0.001, use_gpu=False, iterations=1000, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6039 sparse matrix of type '<class 'numpy.int64'>'
	with 836481 stored elements in Compressed Sparse Column format>

In [20]:
als_model.fit(csr_data_transpose) #모델 학습 

  0%|          | 0/1000 [00:00<?, ?it/s]

## 6) 선호 영화 5가지 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보기


In [31]:
# (num6040) vector와 (back to the future) vector 확인
num6040, backtothefuture = user_to_idx[6040], movie_to_idx['Back to the Future (1985)']
num6040_vector, backtothefuture_vector = als_model.user_factors[num6040], als_model.item_factors[backtothefuture]

In [32]:
num6040_vector

array([-1.7670698 ,  0.9355376 ,  1.8478744 , -2.3997495 , -0.18145958,
        0.8294392 ,  0.94821346,  0.91058725,  1.6605284 ,  0.6130714 ,
        1.2160585 ,  1.4288293 , -1.6243374 , -0.17315306,  1.3435179 ,
        0.33950537,  2.3580163 ,  1.000581  ,  0.62111807,  0.0682307 ,
        1.205535  ,  0.6024656 , -1.87434   , -0.4953582 ,  1.2363542 ,
        0.5304339 ,  2.4603934 , -0.3316388 ,  2.6061985 , -1.2861216 ,
        3.126374  , -3.022511  ,  1.4037706 ,  0.4829077 ,  1.5471526 ,
        0.6896806 , -1.1040905 , -1.2985635 , -0.92768914, -1.2185062 ,
       -2.835399  ,  0.35572752,  3.0007021 , -0.27500284, -1.3564779 ,
       -2.0076158 ,  0.53787404,  2.0309808 , -1.3193716 , -0.5552203 ,
        0.49008432,  1.1500585 , -0.37002963,  2.449063  , -0.6098752 ,
       -1.7316478 ,  0.7912206 ,  1.052556  ,  0.34077924,  1.1908371 ,
       -0.2538377 ,  0.34969872,  1.6705735 ,  1.7549993 ,  1.2167474 ,
        1.3193613 ,  2.6846116 ,  1.3394434 ,  0.13756956,  1.10

In [33]:
backtothefuture_vector

array([ 0.01392102,  0.06913564,  0.01238493,  0.04014229, -0.00547261,
       -0.01913274,  0.01009026,  0.04347604, -0.01870151,  0.04435228,
        0.02767727,  0.01003858, -0.01078627,  0.04700519,  0.05458406,
       -0.01781077, -0.02372367,  0.00945998,  0.03428066, -0.01205594,
       -0.05843735,  0.00165283, -0.0141262 , -0.00628934,  0.01369976,
       -0.03439005,  0.05544429, -0.00332422,  0.0018928 ,  0.01132189,
        0.02348541, -0.00777382,  0.0020961 , -0.01708618,  0.00334208,
       -0.05075391,  0.00092158, -0.05562457, -0.02354231,  0.05584309,
        0.04300875,  0.03041191,  0.00351208, -0.0696599 , -0.01615476,
        0.02495455,  0.02591754,  0.03386325, -0.04027162,  0.00710025,
        0.03734199, -0.04086849, -0.01000024, -0.02891887,  0.04412261,
        0.04194838,  0.01046469,  0.01311885, -0.0390208 ,  0.03178803,
        0.02040835,  0.03735317,  0.0095119 , -0.01866928, -0.02268177,
       -0.00149817,  0.01295832, -0.03213098, -0.01497309, -0.06

In [34]:
#vector을 곱했을 때의 값 확인
np.dot(num6040_vector, backtothefuture_vector)

0.94796425

In [35]:

jumanji = movie_to_idx['Jumanji (1995)']
jumanji_vector = als_model.item_factors[jumanji]

print(np.dot(num6040_vector, jumanji_vector))

0.23923609


## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 보기


In [36]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}

def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [37]:
get_similar_movie('Back to the Future (1985)') 

['Back to the Future (1985)',
 "Ferris Bueller's Day Off (1986)",
 'When Harry Met Sally... (1989)',
 'Back to the Future Part II (1989)',
 'Ghostbusters (1984)',
 'Big (1988)',
 'Bull Durham (1988)',
 'E.T. the Extra-Terrestrial (1982)',
 'Fish Called Wanda, A (1988)',
 'Cocoon (1985)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 보기


In [38]:
user = user_to_idx[6040]
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3125, 1.3072181),
 (1114, 1.1820513),
 (1175, 1.0837772),
 (1071, 1.0647234),
 (2754, 0.9639705),
 (985, 0.96058935),
 (2477, 0.9367366),
 (236, 0.9255878),
 (888, 0.92372584),
 (1844, 0.92343736),
 (1001, 0.8824932),
 (1311, 0.8821777),
 (2462, 0.85646945),
 (2426, 0.85505766),
 (1117, 0.84348726),
 (2074, 0.8151673),
 (2706, 0.80618),
 (1138, 0.78207976),
 (2659, 0.780612),
 (1030, 0.7761719)]

In [39]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Do the Right Thing (1989)',
 'Right Stuff, The (1983)',
 'Gandhi (1982)',
 'Sex, Lies, and Videotape (1989)',
 'Drugstore Cowboy (1989)',
 'Fish Called Wanda, A (1988)',
 '400 Blows, The (Les Quatre cents coups) (1959)',
 'Heavenly Creatures (1994)',
 'African Queen, The (1951)',
 'Roger & Me (1989)',
 'Glengarry Glen Ross (1992)',
 'Kolya (1996)',
 'Ghostbusters (1984)',
 'Buena Vista Social Club (1999)',
 'Sting, The (1973)',
 'Elephant Man, The (1980)',
 'Time Bandits (1981)',
 'Duck Soup (1933)',
 "Ferris Bueller's Day Off (1986)",
 'Jean de Florette (1986)']

## 회고



이 프로젝트에서 indexing을 사용하는데, indexing은 원래의 dataset안에 있는 중복된 요소들을 버리므로써 규모를 축소한 데이터 세트를 만들게 된다. CSR matrix를 만들때는 곱을 하기 때문에 이것으로 CSR matrix를 만든다. 이번 프로젝트에서 indexing의 용도가 CSR matrix를 만들때 너무 커지지 않게 하기 위함인 것 같다. 모델이 추천되는 영화는 내가 좋아하는 영화와 비슷한 것 같기도 하고 나름 흥미로운 것 같다. 